# renaming the reactions according to columns and rows

In [225]:
from math import isclose
from json import load, dump

with open("WesternDietMicrobiomes/escherMap.json", "r") as jsonIn:    escherMap = load(jsonIn)

# display(escherMap)

xs, ys = [], []
for index, lst in enumerate(escherMap):
    for k, content in lst.items():
        if k != "reactions":  continue
        print(content)
        for rxn, content2 in content.items():
            if "label_x" not in content2:  print(content2) ; continue
            if not any([isclose(content2["label_x"], x, abs_tol=50) for x in xs]):   xs.append(content2["label_x"])
            if not any([isclose(content2["label_y"], y, abs_tol=50) for y in ys]):   ys.append(content2["label_y"])

print(xs, ys)

for index, lst in enumerate(escherMap):
    for k, content in lst.items():
        if k != "reactions":  continue
        for rxn, content2 in content.items():
            for x in xs:
                if isclose(content2["label_x"], x, abs_tol=50):
                    col_index = xs.index(x)
                    continue
            for y in ys:
                if isclose(content2["label_y"], y, abs_tol=50):
                    row_index = ys.index(y)
                    continue
            for attribute, val in content2.items():
                if attribute == "metabolites":
                    metabolite_dics = []
                    for metabolite in val:
                        metabolite_dic = {}
                        for cpd_index, (k2, val2) in enumerate(metabolite.items()):
                            if k2 == "bigg_id":  metabolite_dic[k2] = f"{val2}_{col_index}_{row_index}_{cpd_index}"
                            elif k2 == "coefficient":  metabolite_dic[k2] = val2
                        metabolite_dics.append(metabolite_dic)
                    escherMap[index][k][rxn][attribute] = metabolite_dics
                    print(escherMap[index][k][rxn][attribute])
                if attribute == "bigg_id":   escherMap[index][k][rxn][attribute] = f"{val}_{col_index}_{row_index}"
                # if attribute == "metabolites":
                    # escherMap[index][k][rxn][attribute] = val+
                # print(attribute, val)


with open("WesternDietMicrobiomes/escherMap_edited.json", "w") as jsonOut:    escherMap = dump(escherMap, jsonOut, indent=3)

FileNotFoundError: [Errno 2] No such file or directory: 'WesternDietMicrobiomes/escherMap.json'

# duplicating reactions across a specified number of rows and columns

In [ ]:
2961.18779884681--638.8122011531899

3600.0

In [ ]:
from json import load, dump

def duplicate_rxn(escherJSON_path, rows=None, columns=None, row_step_size=800, col_step_size=400):
    assert rows is not None or columns is not None, "What are you doing?"

    with open(escherJSON_path, "r") as jsonIn:
        escherMap = load(jsonIn)

    # display(escherMap)
    print(len(escherMap))

    print(row_step_size, col_step_size)

    for i, lst in enumerate(escherMap):
        if i == 0:  continue
        for k, content in lst.items():
            if k == "reactions":
                # row reaction development
                print("rows")
                print(row_step_size, col_step_size)
                updated_reactions = {} if "updated_reactions" not in locals() else updated_reactions
                if isinstance(rows, int):
                    row_reactions = {k:v for k,v in content.items()}
                    for row_index in range(1, rows):
                        # print(content.keys())
                        for rxnNum, content2 in content.items():
                            nextRxn = str(int(rxnNum)+1)
                            ## define the content of this reaction
                            new_content = {k:v for k,v in content2.items()}
                            ## translate the label
                            new_content["label_y"] += row_step_size*row_index
                            ## rename the reaction according to its new translation
                            rxnName_content = new_content["bigg_id"].split("_")
                            colNum, rowNum = rxnName_content[1], rxnName_content[2]
                            new_content["bigg_id"] = f"{rxnName_content[0]}_{colNum}_{str(int(rowNum)+1)}"
                            ## rename the metabolites
                            for index, metabolite in enumerate(new_content["metabolites"]):
                                ### rename the metabolites
                                ID_content = metabolite["bigg_id"].split("_")
                                metabolite["bigg_id"] = f"{ID_content[0]}_{colNum}_{rowNum}_{index}"
                            ## update the metabolite nodes
                            display(new_content.keys())
                            ## update the edge segments
                            for num, content3 in new_content["segments"].items():
                                dictionary_values = [value for value in content3.values() if isinstance(value, dict)]
                                # print(dictionary_values)
                                # for obj, value in content3.items():
                                #     # print(list(map(len, [new_content, content3])))
                                #     if isinstance(value, dict):
                                #         print(rxnNum, num, obj, value) ;  new_content["segments"][num][obj]["y"] += row_step_size*row_index
                                #     if isinstance(value, dict):
                                #         print(new_content["segments"][num][obj])
                            row_reactions[str(int(rxnNum)+row_index)] = new_content
                            # print(row_reactions[str(int(rxn)+row_index)]["segments"]["312"]["b2"])
                        updated_reactions.update({k:v for k,v in row_reactions.items()})
                escherMap[i][k] = updated_reactions  
            elif k == "nodes":
                # row metabolite development
                print("rows")
                print(row_step_size, col_step_size)
                updated_reactions = {} if "updated_reactions" not in locals() else updated_reactions
                if isinstance(rows, int):
                    row_nodes = {k:v for k,v in content.items()}
                    for row_index in range(1, rows):
                        print(content.keys())
                        # updated_reactions = {} if "updated_reactions" not in locals() else updated_reactions
                        new_mets = {}
                        for metNum, content2 in row_nodes.items():
                            ### rename the node for this set of reactions
                            # print(metNum)
                            new_mets[f"{nextRxn}{metNum}"] = content2
                            ### translating the node locations
                            if new_mets[f"{nextRxn}{metNum}"]["node_type"] in ["multimarker", "midmarker"]:
                                new_mets[f"{nextRxn}{metNum}"]["y"] += row_step_size*row_index
                            elif new_mets[f"{nextRxn}{metNum}"]["node_type"] == "metabolite":
                                new_mets[f"{nextRxn}{metNum}"]["y"] += row_step_size*row_index
                                new_mets[f"{nextRxn}{metNum}"]["label_y"] += row_step_size*row_index
                        updated_reactions.update({k:v for k,v in row_reactions.items()})
                escherMap[i][k] = updated_reactions  
            # else:   print(k)
        # if index == len(escherMap)-1:  break
        # print("columns")
        # print(row_step_size, col_step_size)
        # if isinstance(columns, int):
        #     reactions = content if rows is not None else updated_reactions
        #     col_reactions = {}
        #     for col_index in range(columns):
        #         for rxnNum, content2 in reactions.items():
        #             new_content = {k:v for k,v in content2.items()}
        #             new_content["label_x"] += col_step_size*col_index
        #             for num, content3 in new_content["segments"].items():
        #                 for obj, value in content3.items():
        #                     if isinstance(value, dict):  print(rxnNum, num, obj, value) ; new_content["segments"][num][obj]["x"] += col_step_size*col_index
        #                     if isinstance(value, dict):  print(new_content["segments"][num][obj])
        #             col_reactions[str(int(rxnNum)+col_index)] = new_content
        #         updated_reactions.update({k:v for k,v in row_reactions.items()})

        #     escherMap[index][k] = updated_reactions    

    with open(escherJSON_path.replace(".json", "_edited.json"), "w") as jsonOut:  dump(escherMap, jsonOut, indent=3)

duplicate_rxn("oneRXN_localized2.json", rows=3, columns=None)

2
800 400
rows
800 400


dict_keys(['name', 'bigg_id', 'reversibility', 'label_x', 'label_y', 'gene_reaction_rule', 'genes', 'metabolites', 'segments'])

dict_keys(['name', 'bigg_id', 'reversibility', 'label_x', 'label_y', 'gene_reaction_rule', 'genes', 'metabolites', 'segments'])

rows
800 400
dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22'])
dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22'])


In [ ]:
print(1596.7619819523752-396.7619819523753)
print(2796.761981952375-1596.7619819523752)

1200.0
1200.0


In [ ]:
from json import load, dump

path = "oneRXN_localized_double.json"

with open(path, "r") as jsonIn:
    escherMap = load(jsonIn)

with open(path, "w") as jsonOut:
    dump(escherMap, jsonOut, indent=3)

## update the coordinates of an existing two-reaction map

In [ ]:
from json import load

with open("twoRXN_localized.json", 'r') as jsonIn:
    EscherMap = load(jsonIn)

row_step_size, col_step_size = 800, 400
column = 0

new_EscherMap = {}
for i, ele in enumerate(EscherMap):
    if i == 0:
        new_EscherMap[i] = ele
    new_EscherMap[i] = {}
    for k, content in ele.items():
        # print(k)
        if k == "reactions":
            new_EscherMap[i]["reactions"] = {}
            for rxnNum, content2 in content.items():
                if rxnNum == 0:
                    new_EscherMap[i]["reactions"][rxnNum] = content2
                    continue
                for k2, val in content2.items():
                    if k == "bigg_id":
                        baseRXN = val.split("_")[0]
                        new_EscherMap[i]["reactions"][rxnNum][k2] = f"{baseRXN}_{column}_{rxnNum}"
                    elif k == "label_x":
                        new_EscherMap[i]["reactions"][rxnNum][k2] = val + column*col_step_size
                    elif k == "label_y":
                        new_EscherMap[i]["reactions"][rxnNum][k2] = val + rxnNum*row_step_size
                    elif k2 == "metabolites":
                        new_mets = []
                        for cpdNum, cpd in enumerate(val):
                            cpd["bigg_id"] = f"cpd_{column}_{rxnNum}_{cpdNum}"
                            new_mets.append(cpd)
                        new_EscherMap[i]["reactions"][rxnNum][k2] = new_mets
                    elif k2 == "segments":
                        for segNum, content3 in val.items():
                            if isinstance(content3["b1"], dict):
                                new_EscherMap[i]["reactions"][rxnNum][k2][segNum][]
                    


                display(content2)
                break

        elif k == "nodes":
            pass
        elif k == "text_labels":
            pass
        elif k == "canvas":
            pass

{'name': 'diphosphate phosphohydrolase',
 'bigg_id': 'rxn00001_0_0',
 'reversibility': False,
 'label_x': -682.8153489006918,
 'label_y': 228.44559706520863,
 'gene_reaction_rule': '',
 'genes': [],
 'metabolites': [{'bigg_id': 'cpd_0_0_0', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_1', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_2', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_3', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_4', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_5', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_6', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_7', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_8', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_9', 'coefficient': -1},
  {'bigg_id': 'cpd_0_0_10', 'coefficient': 1},
  {'bigg_id': 'cpd_0_0_11', 'coefficient': 1},
  {'bigg_id': 'cpd_0_0_12', 'coefficient': 1},
  {'bigg_id': 'cpd_0_0_13', 'coefficient': 1},
  {'bigg_id': 'cpd_0_0_14', 'coefficient': 1},
  {'bigg_id': 'cpd_0_0_15', 'coefficient': 1},
  {'bigg_id': 'cpd_0_0_16', 'coeff

# process the ASVMetaboliteInteractions community interactions

## parse the stoichiometric relationships

In [ ]:
from pandas import read_csv
from json import dump

interactions = read_csv("ASVMetaboliteInteractions.csv").set_index("ASV")
stoich_mapping = {}
for index, row in interactions.iterrows():
    stoich_mapping[index] = {"reactants":{}, "products":{}}
    for col, ele in row.items():
        metID = col.split("_")[0]
        if ele > 0:   stoich_mapping[index]["products"][metID] = ele
        elif ele < 0:   stoich_mapping[index]["reactants"][metID] = ele

# display(stoich_mapping)
with open("ASV_metabolic_activity_stoich.json", 'w') as jsonOut:
    dump(stoich_mapping, jsonOut, indent=3)

# parse the model for content and interactions

In [254]:
from cobra.io import load_json_model
from optlang import Objective

effectiveASV_model = load_json_model("ASVInteractionModel.json")
rxnNames = [rxn.id for rxn in effectiveASV_model.reactions]
rcNames = {}
numRCs = []
for name in rxnNames:
    if "RC" in name:
        nameParts = name.split("RC")
        numRCs.append(nameParts[0])
        rcNames[nameParts[0]] = rcNames.get(nameParts[0], [])
        rcNames[nameParts[0]].append(nameParts[1].split("_")[1])
# print(set(rcNames.keys()) - set(numRCs))
wdNames = {}
numWDs = []
for name in rxnNames:
    if "WD" in name:
        nameParts = name.split("WD")
        numWDs.append(nameParts[0])
        wdNames[nameParts[0]] = wdNames.get(nameParts[0], [])
        wdNames[nameParts[0]].append(nameParts[1].split("_")[1])
# print(set(wdNames.keys()) - set(numWDs))
new_rcNames = [f"{k}RC-ABX_{time}" for k,v in rcNames.items() for time in v]
new_wdNames = [f"{k}WD-ABX_{time}" for k,v in wdNames.items() for time in v]

missing_keys = set(rxnNames) - set(new_rcNames+new_wdNames)
# print(missing_keys)


#define the categories of compounds that will be examined
compounds = {"Simple_carbs": ["Dulcose", "Galactose", "D-Fructose", "D-Allose", "L-Arabinose", "D-Glucose", "D-Ribose",
                               "L-Inositol", "Sucrose", "L-Fucose", "Thyminose", "L-Rhamnose"],
             "Complex_carbs": ["Melibiose", "Melitose", "CELB"],
             "SCFAs": ["Acetate", "Propionate", "Butyrate"],
             "Fatty Acids": ["Decanoate", "Myristic acid", "Palmitate"],
             "Amino Acids": ["L-Methionine", "L-Histidine", "L-Tryptophan", "L-Alanine", "L-Threonine", "L-Glutamate", "L-Isoleucine", "L-Proline", "L-Leucine", "Glycine", 'L-Phenylalanine',
                             'L-Tyrosine', 'L-Valine', 'L-Glutamine', 'L-Arginine', 'L-Asparagine', 'L-Aspartate', 'L-Lysine', 'L-Serine', 'L-Cysteine', "D-Serine", "5-Oxoproline"],
             "Simple_carbon":["Succinate", "Citrate", "L-Malate", "Pyruvate"],
             "Other": ["N-Acetyl-D-glucosamine", "Ornithine", "Fumarate", "trans-4-Hydroxy-L-proline", "2-Oxoglutarate", "Niacin"]}


def search_days(day):
    RCasvs = [asv for asv, days in rcNames.items() if day in days]
    WDasvs = [asv for asv, days in wdNames.items() if day in days]
    display("RC", sorted(RCasvs))
    display("WD", sorted(WDasvs))

def biggest_engager(day, diet, compound, all=False, limit=1):
    print(compound)
    if all:
        for rxn in effectiveASV_model.reactions:
            if day in rxn.id and diet in rxn.id:
                for met, coef in rxn.metabolites.items():
                    # print(met.name, coef)
                    if compound == met.name and coef > limit:
                        print(rxn.name)

    maxFlux = [None, 0]
    minFlux = [None, 0]
    for rxn in effectiveASV_model.reactions:
        if day in rxn.id and diet in rxn.id:
            for met, coef in rxn.metabolites.items():
                # print(met.name, coef)
                if compound == met.name:
                    if coef > maxFlux[1]:    maxFlux = [rxn.name, coef]
                    elif coef < minFlux[1]:  minFlux = [rxn.name, coef]
    if maxFlux != [None, 0]:  print("maxFlux", maxFlux)
    if minFlux != [None, 0]:  print("minFlux", minFlux)

def full_assessment(day, diet, compounds, engagers=True, classification=True, skipping_mets=None):
    print(f"metabolite assessment for {day} on the {diet} diet")
    #determine the biggest consumers/producers of each compound
    if engagers:
        for category, metabolites in compounds.items():
            print(category)
            biggest_engager(day, diet, metabolites)

    #determine the compounds that exclusively consumed/produced or both
    if classification:
        metabolites = {"consumed": set(), "produced": set(), "intermediates": set()}
        for rxn in effectiveASV_model.reactions:
            if "_"+day in rxn.id and diet in rxn.id:
                for met, coef in rxn.metabolites.items():
                    if skipping_mets and met.name in skipping_mets:  continue
                    if abs(coef) <= 0.05:   continue
                    # print(met.name, coef)
                    if coef > 0:
                        metabolites["produced"].add(met.name)
                        if met.name in metabolites["consumed"]:
                            # print("produced", met.name, rxn.name)
                            metabolites["intermediates"].add(met.name)
                    elif coef < 0:
                        metabolites["consumed"].add(met.name)
                        # print("consumed", met.name, rxn.name)
                        if met.name in metabolites["produced"]:
                            # print("consumed", met.name, rxn.name)
                            metabolites["intermediates"].add(met.name)
        metabolites["consumed"] -= metabolites["intermediates"]
        metabolites["produced"] -= metabolites["intermediates"]
        display(metabolites)



    
def count_interactions(compounds):
    from itertools import product
    days, diets = ["-1.5", "1.5", "4.0", "6.0", "9.0", "12.5"], ["RC", "WD"]
    Interactions = {k:dict() for k in compounds}
    categories = {ele:k for k,v in compounds.items() for ele in v}
    for day, diet in product(days, diets):
        for category in Interactions:
            if day in Interactions[category]:  Interactions[category][day].update({diet:set()})
            else:                    Interactions[category][day] = {diet:set()}
        # for category in InteractionCounts:
        #     if day in InteractionCounts[category]:  InteractionCounts[category][day].update({diet:set()})
        #     else:                    InteractionCounts[category][day] = {diet:set()}
    for day, diet in product(days, diets):
        produced, consumed = set(), set()
        for rxn in effectiveASV_model.reactions:
            metNames = [met.name for met in rxn.metabolites]
            # print(metNames)
            for metName in metNames:
                category = categories[metName]
                if "_"+day in rxn.id and diet in rxn.id:
                    for met, coef in rxn.metabolites.items():
                        # print(met.name, coef)
                        if coef > 0:
                            produced.add(met.name)
                            if met.name in consumed:
                                # print("produced", met.name, rxn.name)
                                Interactions[category][day][diet].add(met.name)
                        elif coef < 0:
                            consumed.add(met.name)
                            # print("consumed", met.name, rxn.name)
                            if met.name in produced:
                                # print("consumed", met.name, rxn.name)
                                Interactions[category][day][diet].add(met.name)
    # print(aaInteractionsa)
    InteractionCounts = {}
    for category, content in Interactions.items():
        InteractionCounts[category] = {}
        for day, dietInfo in content.items():
            # print(day, InteractionCounts[category])?
            InteractionCounts[category][day] = InteractionCounts[category].get(day, {})
            for diet, mets in dietInfo.items():
                InteractionCounts[category][day][diet] = InteractionCounts[category][day].get(diet, {diet: 0})
                InteractionCounts[category][day][diet] = len(mets)
                # print(day, diet, len(mets)), mets)
    display(InteractionCounts)

    # convert the amino acid counts into a DataFrame
    from pandas import DataFrame
    df = DataFrame(InteractionCounts)
    df.index.name = "Interval Day"
    display(df)
    df.to_csv("InteractionCounts.csv")
    return df

# full_assessment("12.5", "WD", engagers=False, skipping_mets=aa_names+["Acetate", "Niacin"])
InteractionCounts = count_interactions(compounds)

{'Simple_carbs': {'-1.5': {'RC': 23, 'WD': 17},
  '1.5': {'RC': 20, 'WD': 4},
  '4.0': {'RC': 12, 'WD': 0},
  '6.0': {'RC': 8, 'WD': 8},
  '9.0': {'RC': 18, 'WD': 13},
  '12.5': {'RC': 20, 'WD': 11}},
 'Complex_carbs': {'-1.5': {'RC': 17, 'WD': 17},
  '1.5': {'RC': 20, 'WD': 4},
  '4.0': {'RC': 9, 'WD': 0},
  '6.0': {'RC': 12, 'WD': 8},
  '9.0': {'RC': 20, 'WD': 12},
  '12.5': {'RC': 19, 'WD': 10}},
 'SCFAs': {'-1.5': {'RC': 24, 'WD': 17},
  '1.5': {'RC': 18, 'WD': 4},
  '4.0': {'RC': 11, 'WD': 0},
  '6.0': {'RC': 9, 'WD': 8},
  '9.0': {'RC': 14, 'WD': 12},
  '12.5': {'RC': 16, 'WD': 7}},
 'Fatty Acids': {'-1.5': {'RC': 22, 'WD': 17},
  '1.5': {'RC': 20, 'WD': 4},
  '4.0': {'RC': 12, 'WD': 0},
  '6.0': {'RC': 14, 'WD': 8},
  '9.0': {'RC': 20, 'WD': 12},
  '12.5': {'RC': 21, 'WD': 11}},
 'Amino Acids': {'-1.5': {'RC': 25, 'WD': 17},
  '1.5': {'RC': 20, 'WD': 4},
  '4.0': {'RC': 12, 'WD': 0},
  '6.0': {'RC': 14, 'WD': 8},
  '9.0': {'RC': 20, 'WD': 13},
  '12.5': {'RC': 21, 'WD': 11}},
 '

,Simple_carbs,Complex_carbs,SCFAs,Fatty Acids,Amino Acids,Simple_carbon,Other
Interval Day,,,,,,,
-1.5,"{'RC': 23, 'WD': 17}","{'RC': 17, 'WD': 17}","{'RC': 24, 'WD': 17}","{'RC': 22, 'WD': 17}","{'RC': 25, 'WD': 17}","{'RC': 24, 'WD': 17}","{'RC': 25, 'WD': 17}"
1.5,"{'RC': 20, 'WD': 4}","{'RC': 20, 'WD': 4}","{'RC': 18, 'WD': 4}","{'RC': 20, 'WD': 4}","{'RC': 20, 'WD': 4}","{'RC': 19, 'WD': 4}","{'RC': 20, 'WD': 4}"
4.0,"{'RC': 12, 'WD': 0}","{'RC': 9, 'WD': 0}","{'RC': 11, 'WD': 0}","{'RC': 12, 'WD': 0}","{'RC': 12, 'WD': 0}","{'RC': 12, 'WD': 0}","{'RC': 12, 'WD': 0}"
6.0,"{'RC': 8, 'WD': 8}","{'RC': 12, 'WD': 8}","{'RC': 9, 'WD': 8}","{'RC': 14, 'WD': 8}","{'RC': 14, 'WD': 8}","{'RC': 10, 'WD': 8}","{'RC': 14, 'WD': 8}"
9.0,"{'RC': 18, 'WD': 13}","{'RC': 20, 'WD': 12}","{'RC': 14, 'WD': 12}","{'RC': 20, 'WD': 12}","{'RC': 20, 'WD': 13}","{'RC': 19, 'WD': 13}","{'RC': 20, 'WD': 13}"
12.5,"{'RC': 20, 'WD': 11}","{'RC': 19, 'WD': 10}","{'RC': 16, 'WD': 7}","{'RC': 21, 'WD': 11}","{'RC': 21, 'WD': 11}","{'RC': 19, 'WD': 10}","{'RC': 21, 'WD': 11}"


### develop the diverging bar graph of microbiome interactions

In [ ]:
from matplotlib import pyplot

# Plotting the diverging bar chart
fig, ax = pyplot.subplots(figsize=(10, 6))
ax.barh(InteractionCounts['Category'], InteractionCounts['Positive'], color='g', label='Positive')
ax.barh(InteractionCounts['Category'], InteractionCounts['Negative'], color='r', label='Negative')

# curtail the figure
x_max = max(InteractionCounts['Positive'].max(), -InteractionCounts['Negative'].min())
ax.set_xlim(-x_max, x_max)
ax.set_xlabel('Values')
ax.set_ylabel('Categories')
ax.set_title('Double-ended Bar Graph')
ax.legend()

# Add grid lines for better readability
pyplot.grid(True, axis='x', linestyle='--', alpha=0.7)
pyplot.savefig("diverging_barChart.png")

In [374]:
from itertools import product

days, diets = ["-1.5", "1.5", "4.0", "9.0", "12.5"], ["RC", "WD"]
print([x for x in product(days, diets)])

[('-1.5', 'RC'), ('-1.5', 'WD'), ('1.5', 'RC'), ('1.5', 'WD'), ('4.0', 'RC'), ('4.0', 'WD'), ('9.0', 'RC'), ('9.0', 'WD'), ('12.5', 'RC'), ('12.5', 'WD')]


# define new metabolite categories

In [ ]:
from modelseedpy.biochem import from_local

msdb = from_local("../../ModelSEEDDatabase")

In [ ]:
# from deepdiff import DeepDiff
from numpy import array, load, append, save, concatenate, setdiff1d

scfa = load("../../CommScores/commscores/data/categories/scfa.npy")
vitamins = load("../../CommScores/commscores/data/categories/vitamins.npy")
aminoAcids = load("../../CommScores/commscores/data/categories/aminoAcids.npy")
sugars = load("../../CommScores/commscores/data/categories/sugars.npy")
# complex_sugars = array([["cpd00158", "CELB"], ["cpd00382", "Melitose"]])
# save("../../CommScores/commscores/data/categories/complex_sugars.npy", complex_sugars)
# sugars_dict = dict(sugars)
# new_sugars_dict = {}
# for cpdID, name in sugars_dict.items():
#     if name not in ["CELB", "Melitose"]:
#         new_sugars_dict[cpdID] = name
# sugars = array(list(zip(new_sugars_dict.keys(), new_sugars_dict.values())))
# save("../../CommScores/commscores/data/categories/sugars.npy", sugars)
# print(DeepDiff(sugars_dict, new_sugars_dict))
complex_sugars = load("../../CommScores/commscores/data/categories/complex_sugars.npy")
# new_sugars = setdiff1d(sugars, complex_sugars)
# display(new_sugars)
# new_sugars = subtract(sugars, array([["cpd00158", "CELB"], ["cpd00382", "Melitose"]]))
# display(new_sugars)
# save("../../CommScores/commscores/data/categories/sugars.npy", new_sugars)
energy_compounds = load("../../CommScores/commscores/data/categories/energy_compounds.npy")

In [ ]:
from numpy import save, delete
fatty_acids = [["cpd01113", "Hexanoate"], ["cpd03846", "Octanoate"], ["cpd01107", "Caprate"], ["cpd01741", "Laurate"],
               ["cpd15640", "Tridecanoate"], ["cpd03847", "Myristate"], ["cpd16351", "Pentadecanoate"], ["cpd00214", "Palmitate"],
               ["cpd15609", "heptadecanoate"], ["cpd01080", "Stearate"], ["cpd00536", "Oleate"],
               ["cpd01122", "Linoelate"], ["cpd03850", "Linoleate"], ["cpd03848", "Arachidate"]]
save("../../CommScores/commscores/data/categories/fattyAcids.npy", fatty_acids)

In [ ]:
display(aminoAcids)

array([['cpd00035', 'Alanine'],
       ['cpd00051', 'L-Arginine'],
       ['cpd00132', 'L-Asparagine'],
       ['cpd00041', 'L-Aspartate'],
       ['cpd00084', 'L-Cysteine'],
       ['cpd00023', 'L-Glutamate'],
       ['cpd00053', 'L-Glutamine'],
       ['cpd00033', 'Glycine'],
       ['cpd00119', 'L-Histidine'],
       ['cpd00747', 'L-Hydroxyproline'],
       ['cpd00322', 'L-Isoleucine'],
       ['cpd00107', 'L-Leucine'],
       ['cpd00039', 'L-Lysine'],
       ['cpd00060', 'L-Methionine'],
       ['cpd00066', 'L-Phenylalanine'],
       ['cpd00129', 'L-Proline'],
       ['cpd00054', 'L-Serine'],
       ['cpd00161', 'L-Threonine'],
       ['cpd00065', 'L-Tryptophan'],
       ['cpd00069', 'L-Tyrosine'],
       ['cpd00156', 'L-Valine']], dtype='<U16')

In [ ]:
# define the sets of amino acids
aromaticAA = array([['cpd00066', 'L-Phenylalanine'], ['cpd00069', 'L-Tyrosine'], ['cpd00065', 'L-Tryptophan']])
pyruvateAA = array([['cpd00035', 'Alanine'], ['cpd00107', 'L-Leucine'], ['cpd00322', 'L-Isoleucine'], ['cpd00156', 'L-Valine']])
alphaKetaGlutarateAA = array([['cpd00023', 'L-Glutamate'], ['cpd00053', 'L-Glutamine'], ['cpd00129', 'L-Proline'], ['cpd00051', 'L-Arginine'],
                              ['cpd00747', 'L-Hydroxyproline']])
oxaloacetateAA = array([['cpd00132', 'L-Asparagine'], ['cpd00041', 'L-Aspartate'], 
                        ['cpd00060', 'L-Methionine'], ['cpd00161', 'L-Threonine'], ['cpd00039', 'L-Lysine']])
phosphoglycerateAA = array([['cpd00054', 'L-Serine'], ['cpd00033', 'Glycine'], ['cpd00084', 'L-Cysteine'], ["cpd00550", "D-Serine"]])
oxoAAs = array([["cpd01293", "Oxoproline"], ["cpd01292", "Oxoproline"], ["cpd00024", "2-Oxoglutarate"]])

# save the amino acid categories
save("../../CommScores/commscores/data/categories/aromaticAA.npy", aromaticAA)
save("../../CommScores/commscores/data/categories/pyruvateAA.npy", pyruvateAA)
save("../../CommScores/commscores/data/categories/alphaKetaGlutarateAA.npy", alphaKetaGlutarateAA)
save("../../CommScores/commscores/data/categories/oxaloacetateAA.npy", oxaloacetateAA)
save("../../CommScores/commscores/data/categories/phosphoglycerateAA.npy", phosphoglycerateAA)
save("../../CommScores/commscores/data/categories/oxoAAs", oxoAAs)

In [ ]:
from numpy import concatenate

all_specifiedAAs = concatenate([aromaticAA, pyruvateAA, alphaKetaGlutarateAA, oxaloacetateAA, phosphoglycerateAA])
for aa in aminoAcids:
    if aa not in all_specifiedAAs:
        print(aa)

['cpd00119' 'L-Histidine']


In [ ]:
scfa
# new_energy_compounds = []
# for cpd, name in energy_compounds:
#     # print(cpd, name)
#     if name in fatty_acids:
#         new_energy_compounds = delete(energy_compounds, name, 1)

array([['cpd00029', 'Acetate'],
       ['cpd00141', 'Propionate'],
       ['cpd00211', 'Butyrate'],
       ['cpd01711', 'Isobutyrate'],
       ['cpd00597', 'Valerate'],
       ['cpd16265', 'M_2_Keto_isovalerate'],
       ['cpd35499', '(S)-2-methylbutanoate'],
       ['cpd00106', 'Fumarate']], dtype='<U21')

In [ ]:
print(len(new_energy_compounds), len(energy_compounds))

0 29614


In [ ]:
new_scfa = append(scfa, [["cpd00106", "Fumarate"]], axis=0)
display(new_scfa)

array([['cpd00029', 'Acetate'],
       ['cpd00141', 'Propionate'],
       ['cpd00211', 'Butyrate'],
       ['cpd01711', 'Isobutyrate'],
       ['cpd00597', 'Valerate'],
       ['cpd16265', 'M_2_Keto_isovalerate'],
       ['cpd35499', '(S)-2-methylbutanoate'],
       ['cpd00106', 'Fumarate']], dtype='<U21')

## categorizing compounds

In [ ]:
from json import load

with open("ASVInteractionModel.json", 'r') as jsonIn:
    model = load(jsonIn)

# check which vitamins are in the model
# metabolites = [met["id"] for met in model["metabolites"]]
# vitamin_dict = dict(vitamins)
# for metID in metabolites:
#     if metID in vitamin_dict:
#         print(metID)

# define dictionaries of all the categories
## amino acids
aromaticAA_dict, pyruvateAA_dict, alphaKetaGlutarateAA_dict = dict(aromaticAA), dict(pyruvateAA), dict(alphaKetaGlutarateAA)
oxaloacetateAA_dict, phosphoglycerateAA_dict, oxoAA_dict = dict(oxaloacetateAA), dict(phosphoglycerateAA), dict(oxoAAs)
## fatty acids
fatty_acids_dict, scfa_dict, sugars_dict, complex_sugars_dict = dict(fatty_acids), dict(scfa), dict(sugars), dict(complex_sugars)

# categorize the model metabolites
not_changed = []
for met in model["metabolites"]:
    ## amino acids
    if met["id"] in aromaticAA_dict:   met["id"] = "Aromatic AA"
    elif met["id"] in pyruvateAA_dict:   met["id"] = "Pyruvate AA"
    elif met["id"] in alphaKetaGlutarateAA_dict:   met["id"] = "alphaKetaGlutarate AA"
    elif met["id"] in oxaloacetateAA_dict:   met["id"] = "Oxaloacetate AA"
    elif met["id"] in phosphoglycerateAA_dict:   met["id"] = "phosphoGlycerate AA"
    elif met["id"] in oxoAA_dict:   met["id"] = "oxoAA"
    ## other compounds
    elif met["id"] in fatty_acids_dict:   met["id"] = "fatty Acid"
    elif met["id"] in scfa_dict:   met["id"] = "scFatty Acid"
    elif met["id"] in sugars_dict:   met["id"] = "Sugar"
    elif met["id"] in complex_sugars_dict:   met["id"] = "complex Sugar"
    else:   not_changed.append(met["id"])

print(f"{len(not_changed)}/{len(model['metabolites'])} compounds are not yet exchanged")
print(not_changed)

# export the converted model
from json import dump
print(model["metabolites"])
with open("ASVInteractionModel_categorizedIDs.json", 'w') as jsonOut:
    dump(model, jsonOut, indent=3)

18/63 compounds are not yet exchanged
['cpd35610', 'cpd00851', 'cpd07053', 'cpd02475', 'cpd04945', 'cpd00122', 'cpd00526', 'cpd00137', 'cpd02733', 'cpd00119', 'cpd00359', 'cpd00121', 'cpd00130', 'cpd00392', 'cpd00218', 'cpd00064', 'cpd00020', 'cpd00036']
[{'id': 'cpd35610', 'name': '3alpha-hydroxy-7-oxo-5beta-cholan-24-oate', 'compartment': ''}, {'id': 'cpd00851', 'name': 'trans-4-Hydroxy-L-proline', 'compartment': ''}, {'id': 'oxoAA', 'name': '5-Oxoproline', 'compartment': ''}, {'id': 'cpd07053', 'name': 'Picolinic acid', 'compartment': ''}, {'id': 'cpd02475', 'name': 'Lithocholate', 'compartment': ''}, {'id': 'cpd04945', 'name': 'Ursodiol', 'compartment': ''}, {'id': 'scFatty Acid', 'name': 'Acetate', 'compartment': ''}, {'id': 'cpd00122', 'name': 'N-Acetyl-D-glucosamine', 'compartment': ''}, {'id': 'oxoAA', 'name': '2-Oxoglutarate', 'compartment': ''}, {'id': 'Pyruvate AA', 'name': 'L-Alanine', 'compartment': ''}, {'id': 'Sugar', 'name': 'D-Allose', 'compartment': ''}, {'id': 'Sugar

# filtering the Escher map for fewer reactions based on the metabolite fluxes

In [357]:
aa_names = ["L-Methionine", "L-Histidine", "L-Tryptophan", "L-Alanine", "L-Threonine", "L-Glutamate", "L-Isoleucine", "L-Proline", "L-Leucine", "Glycine",
 'L-Phenylalanine', 'L-Tyrosine', 'L-Valine', 'L-Glutamine', 'L-Arginine', 'L-Asparagine', 'L-Aspartate', 'L-Lysine', 'L-Serine', 'L-Cysteine', "D-Serine",
 "5-Oxoproline"]

In [359]:
from json import load, dump
with open("metabolite_focused_map.json", 'r') as jsonIn:
    escherMap = load(jsonIn)
# display(escherMap)

og_escherMap = [ele for ele in escherMap]

# from collections import Counter
nodeID = {nodeNum: content["bigg_id"] for nodeNum, content in og_escherMap[1]["nodes"].items() if "bigg_id" in content}
with open("Escher_nodeNum_cpdIDs.json", 'w') as jsonOut:
    dump(nodeID, jsonOut, indent=3)

idName = {content["bigg_id"]: content["name"] for content in og_escherMap[1]["nodes"].values() if "bigg_id" in content}
with open("EscherNodeMapping.json", 'w') as jsonOut:
    dump(idName, jsonOut, indent=3)

newEscher = []
for index, ele in enumerate(og_escherMap):
    og_ele = {k:v for k,v in ele.items()}
    if index == 0:  newEscher = [ele]
    newEscher.append({})
    nodes_to_remove = []
    nodeNumCoef = {}
    for k, val in ele.items():
        if k == "reactions":
            newEscher[index][k] = val
            # newEscher[index][k] = {}
            # reactions = {}
            for rxnNum, content in val.items():
                # display(len(content["metabolites"]), content)
                # print(len(content["metabolites"]))
                #create a mapping from IDs to their respective coefficients
                metCoefs = {met["bigg_id"]: met["coefficient"] for met in content["metabolites"]}
                nameID = {}
                # print(metCoefs)
                for ID in metCoefs.keys():
                    if ID not in idName:   continue
                    name = idName[ID]
                    if name in nameID: nameID[name].append(ID)
                    else: nameID[name] = [ID]
                #determine the nodes that should be removed
                nodeNumIDs = {content2["to_node_id"]: nodeID[content2["to_node_id"]]
                              for content2 in content["segments"].values() if content2["to_node_id"] in nodeID.keys()}
                nodeNumIDs.update({content2["from_node_id"]: nodeID[content2["from_node_id"]]
                                   for content2 in content["segments"].values() if content2["from_node_id"] in nodeID.keys()})
                nodeNumCoef.update({num: metCoefs[ID] for num, ID in nodeNumIDs.items()})
                # print(nodeNumCoef)
                # for nodeNum, coef in nodeNumCoef.items():
                #     if coef < 0.01:   nodes_to_remove.append(nodeNum)
                #     elif "AA" in idName[content["bigg_id"]] and metCoefs[content["bigg_id"]] < 0.1:  continue
                #     for k2, v in content.items():
                #         if k2 == "node_is_primary" and metCoefs[content["bigg_id"]] >= 1:
                #             new_nodes[nodeNum][k2] = True
                #         else:  new_nodes[nodeNum][k2] = v

                # names = Counter([idName[ID]:ID for ID in metCoefs.keys()])
                #determine the ID with the highest coefficient, which will be exclusively kept in the final Escher
                # new_met_content = []
                # for k2,v in nameID.items():
                #     if metCoefs[content["bigg_id"]] < 0.01:   continue
                #     elif "AA" in idName[content["bigg_id"]] and metCoefs[content["bigg_id"]] < 0.1:  continue
                #     for k2, v in content.items():
                #         if k2 == "node_is_primary" and metCoefs[content["bigg_id"]] >= 1:
                #             new_nodes[nodeNum][k2] = True
                #         else:  new_nodes[nodeNum][k2] = v
                #     if len(v) == 1:
                #         new_met_content.append({"bigg_id": v[0], "coefficient": metCoefs[v[0]]})
                #     elif len(v) > 1:
                #         ReactantWinner, ProductWinner = [None, 0], [None, 0]
                #         for ID, coef in metCoefs.items():
                #             if ID in v:
                #                 if coef > 0 and coef > ProductWinner[1]:
                #                     ProductWinner = [ID, coef]
                #                 if coef < 0 and coef < ReactantWinner[1]:
                #                     ReactantWinner = [ID, coef]
                #         for met in content["metabolites"]:
                #             if met["bigg_id"] in v and met["bigg_id"] in [ReactantWinner[0], ProductWinner[0]]:
                #                 new_met_content.append(met)
                # print(len(new_met_content), len(content["metabolites"]))
                #update the new Escher Figure
                newEscher[index][k][rxnNum] = content
                # for key, value in content.items():
                #     if key == "metabolites":
                #         newEscher[index][k][rxnNum][key] = new_met_content
                #     else:
                #         newEscher[index][k][rxnNum][key] = value
                # escherMap[index]["reactions"][rxnNum]["metabolites"] = new_met_content
                # print(len(content["metabolites"]) == len(newEscher[index][k][rxnNum]["metabolites"]))
                # display(len(new_met_content), escherMap[index]["reactions"][rxnNum]["metabolites"])
                # break
    # display(nodeNumCoef)
    for k, val in ele.items():
        if k == "nodes":
            new_nodes = {}
            # display(val)
            for nodeNum, content in val.items():
                # display(content)
                new_nodes[nodeNum] = {}
                if nodeNum in nodeNumCoef:
                    # print(nodeID[nodeNum], nodeNumCoef[nodeNum])
                    if abs(nodeNumCoef[nodeNum]) < 0.05:   continue
                    if idName[content["bigg_id"]] in aa_names:  continue # and abs(nodeNumCoef[nodeNum]) < 0.1:  continue
                    if idName[content["bigg_id"]] in ["Niacin", "Acetate"]:  continue
                    # if abs(nodeNumCoef[nodeNum]) >= 1:   primary = True
                for k2, v in content.items():
                    if k2 == "node_is_primary":  new_nodes[nodeNum][k2] = True
                    else:  new_nodes[nodeNum][k2] = v
            # break
            #remove all of the above nodes, which should remove the segments by default
            #replace the IDs with their categories so that the nodes can consolidate
            newEscher[index][k] = new_nodes
            # DeepDiff(val, new_nodes)
        elif k != "reactions":
            newEscher[index][k] = val
    # print(DeepDiff(og_ele, newEscher[index]))

from deepdiff import DeepDiff
# display(DeepDiff(og_escherMap[1], newEscher[1]))

from json import dump
with open("metabolite_focused_map0.json", 'w') as jsonOut:
    dump(newEscher, jsonOut, indent=3)

#make each column an interacting web, which can feed into the next column through some type of overlapping metabolites
#the fatty acids and the sugars will likely be the differentiating factor between the RC and WD regimes
#amino acids and niacin are the insurance policy for syntrophy, and should be used as the structure of the food web 
#group the members who have the same major products, and group seemingly parasitic organisms separately, by programmatically analyzing the highest fluxes for each organism's reactants and products

In [326]:
from collections import Counter
print(Counter(aa_names))

Counter({'L-Alanine': 2, 'L-Methionine': 1, 'L-Tryptophan': 1, 'L-Threonine': 1, 'L-Glutamate': 1, 'L-Isoleucine': 1, 'L-Proline': 1, 'L-Leucine': 1, 'Glycine': 1, 'L-Phenylalanine': 1, 'L-Tyrosine': 1, 'L-Valine': 1, 'L-Glutamine': 1, 'L-Arginine': 1, 'L-Asparagine': 1, 'L-Aspartate': 1, 'L-Lysine': 1, 'L-Serine': 1, 'L-Cysteine': 1})


### manually restore syntrophic compounds

In [314]:
# Turicibacter

info = {"name":"Turicibacter.1","bigg_id":"Turicibacter.1RC-ABX_-1.5","reversibility":False,"label_x":-2050.9305419921875,"label_y":8288.802734375,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00023","coefficient":-0.1040152173141567},{"bigg_id":"cpd00041","coefficient":-0.1042731995047922},{"bigg_id":"cpd00064","coefficient":0.0241392199676796},{"bigg_id":"cpd00084","coefficient":-0.3059013174870795},{"bigg_id":"cpd00108","coefficient":0.3980953511859826},{"bigg_id":"cpd00122","coefficient":-0.225680761223981},{"bigg_id":"cpd00129","coefficient":-0.0200989691680044},{"bigg_id":"cpd00132","coefficient":-0.0185135542972458},{"bigg_id":"cpd00141","coefficient":-0.0158535137273317},{"bigg_id":"cpd00161","coefficient":-0.0233209413247074},{"bigg_id":"cpd00211","coefficient":-0.0817767218820333},{"bigg_id":"cpd00214","coefficient":-0.0129732900159796},{"bigg_id":"cpd00218","coefficient":-0.0016870647208757},{"bigg_id":"cpd00322","coefficient":-0.0337539940226029}],"segments":{"296":{"from_node_id":"313","to_node_id":"314","b1":None,"b2":None},"297":{"from_node_id":"315","to_node_id":"314","b1":None,"b2":None},"298":{"from_node_id":"313","to_node_id":"312","b1":{"x":-1967.361572265625,"y":8418.39892578125},"b2":{"x":-2236.64208984375,"y":8412.42578125}},"299":{"from_node_id":"313","to_node_id":"316","b1":{"x":-1971.361572265625,"y":8418.39892578125},"b2":{"x":-2131.64208984375,"y":8322.42578125}},"300":{"from_node_id":"315","to_node_id":"317","b1":{"x":-1843.361572265625,"y":8418.39892578125},"b2":{"x":-1789.111572265625,"y":8418.39892578125}},"301":{"from_node_id":"313","to_node_id":"318","b1":{"x":-1971.361572265625,"y":8418.39892578125},"b2":{"x":-2146.64208984375,"y":8340.42578125}},"302":{"from_node_id":"315","to_node_id":"319","b1":{"x":-1839.361572265625,"y":8418.39892578125},"b2":{"x":-1834.111572265625,"y":8436.39892578125}},"303":{"from_node_id":"313","to_node_id":"320","b1":{"x":-1971.361572265625,"y":8418.39892578125},"b2":{"x":-2161.64208984375,"y":8358.42578125}},"304":{"from_node_id":"313","to_node_id":"321","b1":{"x":-1971.361572265625,"y":8418.39892578125},"b2":{"x":-2176.64208984375,"y":8376.42578125}},"305":{"from_node_id":"313","to_node_id":"322","b1":{"x":-1971.361572265625,"y":8418.39892578125},"b2":{"x":-2191.64208984375,"y":8394.42578125}},"306":{"from_node_id":"313","to_node_id":"323","b1":{"x":-1971.361572265625,"y":8418.39892578125},"b2":{"x":-2191.64208984375,"y":8430.42578125}},"307":{"from_node_id":"313","to_node_id":"324","b1":{"x":-1971.361572265625,"y":8418.39892578125},"b2":{"x":-2176.64208984375,"y":8448.42578125}},"308":{"from_node_id":"313","to_node_id":"325","b1":{"x":-1971.361572265625,"y":8418.39892578125},"b2":{"x":-2161.64208984375,"y":8466.42578125}},"309":{"from_node_id":"313","to_node_id":"326","b1":{"x":-1971.361572265625,"y":8418.39892578125},"b2":{"x":-2146.64208984375,"y":8484.42578125}},"310":{"from_node_id":"313","to_node_id":"327","b1":{"x":-1971.361572265625,"y":8418.39892578125},"b2":{"x":-2131.64208984375,"y":8502.42578125}},"311":{"from_node_id":"313","to_node_id":"328","b1":{"x":-1971.361572265625,"y":8418.39892578125},"b2":{"x":-2116.64208984375,"y":8520.42578125}}}}

In [315]:
for num, content in info["segments"].items():
    if "from_node_id" not in content:  continue
    # print(content["from_node_id"], content["to_node_id"])
    print(nodeID[content["from_node_id"]] if content["from_node_id"] in nodeID else "None")
    print((content["to_node_id"], nodeID[content["to_node_id"]], idName[nodeID[content["to_node_id"]]]) if content["to_node_id"] in nodeID else "None")

None
None
None
None
None
('312', 'cpd00023', 'L-Glutamate')
None
('316', 'cpd00041', 'L-Aspartate')
None
('317', 'cpd00064', 'Ornithine')
None
('318', 'cpd00084', 'L-Cysteine')
None
('319', 'cpd00108', 'Galactose')
None
('320', 'cpd00122', 'N-Acetyl-D-glucosamine')
None
('321', 'cpd00129', 'L-Proline')
None
('322', 'cpd00132', 'L-Asparagine')
None
('323', 'cpd00141', 'Propionate')
None
('324', 'cpd00161', 'L-Threonine')
None
('325', 'cpd00211', 'Butyrate')
None
('326', 'cpd00214', 'Palmitate')
None
('327', 'cpd00218', 'Niacin')
None
('328', 'cpd00322', 'L-Isoleucine')


# editing the SVG Escher Map

In [69]:
from modelseedpy.biochem import from_local
msdb = from_local("../../ModelSEEDDatabase")

modelseedpy 0.2.2


Metabolite identifier,cpd00027
Name,D-Glucose
Memory address,0x14a6602b0
Formula,C6H12O6
Compartment,
In 0 reaction(s),


In [74]:
cpd = msdb.compounds.get_by_id("cpd00027")
print(dir(cpd))
print(cpd.annotation["BiGG"])

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_annotation', '_bound', '_id', '_model', '_reaction', '_repr_html_', '_set_id_with_model', 'abbr', 'annotation', 'charge', 'compartment', 'constraint', 'copy', 'delta_g', 'delta_g_error', 'elements', 'flags', 'formula', 'formula_weight', 'id', 'inchi', 'inchi_key', 'is_abstract', 'is_cofactor', 'is_core', 'is_obsolete', 'linked_compound', 'mass', 'model', 'name', 'names', 'notes', 'pka', 'pkb', 'reactions', 'remove_from_model', 'seed_id', 'shadow_price', 'smiles', 'source', 'summary', 'to_template_compartment_compound', 'y']
{'glc__D'}


In [216]:
from pandas import DataFrame

nameAbbrev = DataFrame(columns=["name", "abbrev"])

cpdID_abbrv = {
    'cpd00076': 'sucr', 'cpd00141': 'prpa', 'cpd00211': 'butr', 'cpd00130': 'malat', 'cpd00137': 'citr', 'cpd00024': 'akg', 
    'cpd00036': 'succ', 'cpd00064': 'ornth', 'cpd03847': 'myrst', 'cpd00106': 'fumr', 'cpd00108': 'galct', 'cpd00382': 'melit', 
    'cpd00105': 'ribs', 'cpd00121': 'inost', 'cpd00751': 'fucos', 'cpd00122': 'acglum', 'cpd00158': 'cellb', 
    'cpd00214': 'palm', 'cpd03198': 'melib', 'cpd01171': 'dulco', 'cpd00020': 'pyr', 'cpd00224': 'arbns', 'cpd00082': 'fru', 
    'cpd00851': '4hpro', 'cpd01107': 'decac', 'cpd00396': 'rhmn', 'cpd01055': 'allos', 'cpd00027': 'glu', 
    'cpd01242': '2drib'}

for cpdID, abbrev in cpdID_abbrv.items():
    cpd = msdb.compounds.get_by_id(cpdID)
    nameAbbrev.loc[len(nameAbbrev)] = [cpd.name, abbrev]

display(nameAbbrev)
print(len(cpdID_abbrv))

,name,abbrev
0,Sucrose,sucr
1,Propionate,prpa
2,Butyrate,butr
3,L-Malate,malat
4,Citrate,citr
5,2-Oxoglutarate,akg
6,Succinate,succ
7,Ornithine,ornth
8,Myristic acid,myrst
9,Fumarate,fumr


29


In [228]:
# from shapely.geometry import Point, box
import random
import re

def tint_color(hex_color, tint_factor=0.5):
    rgb_color = tuple(int(hex_color.lstrip('#')[i:i+2], 16) for i in (0, 2, 4))
    tinted_rgb = [((1 - tint_factor) * component + tint_factor * 255) for component in rgb_color]
    return '{:02x}{:02x}{:02x}'.format(int(tinted_rgb[0]), int(tinted_rgb[1]), int(tinted_rgb[2]))

# Function to get bounding box of an SVG element
def get_text_bbox(text_element, font_size=10):
    x = float(text_element['x'])
    y = float(text_element['y'])
    # Approximate width and height based on font size
    width = len(text_element.get_text()) * font_size * 0.6
    height = font_size
    return box(x, y - height, x + width, y)

# Function to approximate the bounding box of a node element
def get_node_bbox(node_element):   # TODO need to be updated
    cx = float(node_element['cx'])
    cy = float(node_element['cy'])
    r = float(node_element['r'])
    return box(cx - r, cy - r, cx + r, cy + r)

# Function to approximate the bounding box of an edge element (assuming it's a line)
def get_edge_bbox(edge_element):   # TODO need to be updated
    x1 = float(edge_element['x1'])
    y1 = float(edge_element['y1'])
    x2 = float(edge_element['x2'])
    y2 = float(edge_element['y2'])
    # A simple approximation of the bounding box of a line
    return box(min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2))


# Function to adjust text labels to minimize overlap
def adjust_labels(labels, nodes, edges, font_size=10, textToKeep=["l27", "l13"]):
    label_positions = [get_text_bbox(label, font_size) for label in labels if label["id" not in textToKeep]]
    node_positions = [get_node_bbox(node) for node in nodes]
    edge_positions = [get_edge_bbox(edge) for edge in edges]
    
    all_positions = label_positions + node_positions + edge_positions
    for i, label in enumerate(labels):
        bbox = label_positions[i]
        adjusted = False
        
        while not adjusted:
            overlap = False
            for j, other_bbox in enumerate(all_positions):
                if i != j and bbox.intersects(other_bbox):
                    overlap = True
                    break
            
            if overlap:
                # Adjust position (for simplicity, moving randomly; a better algorithm can be used)
                match = re.search(r'translate\((-?\d+\.?\d*),\s*(-?\d+\.?\d*)\)', label['transform'])
                x, y = float(match.group(1)), float(match.group(2))
                x = str(x + random.uniform(-1, 1))
                y = str(y + random.uniform(-1, 1))
                label["transform"] = f'translate({x},{y})'
                bbox = get_text_bbox(label, font_size)
                label_positions[i] = bbox
            else:     adjusted = True


def EscherSVG_processing(svg_path="metabolite_focused_map.svg", labels_to_remove=["stoichiometry-labels"],
                         node_label_px=10, rxn_label_px=30, node_label_hex="e0865b", rxn_label_hex="202078",
                         rxn_edge_hex="5aa6df", rxn_edge_px=10, colorElements=None, largeNodeLabels=None,
                         largeEdgeLabels=None, tint_factor=0.5, abbrevIDs=None, msdb=None):
    # load the SVG for parsing
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(open(svg_path, "r").read() , 'lxml-xml')

    if abbrevIDs:
        for nodeLabel in soup.find_all('text', class_="node-label label"):
            cpdID = nodeLabel.string
            nodeLabel.string = abbrevIDs[cpdID]

    # remove all stoichiometric or other undesirable labels
    for label in labels_to_remove:
        stoich_labels = soup.find_all(class_=label)
        for label in stoich_labels:
            label.decompose()

    # label font-size and color change
    metadata = soup.find("style", attrs={'type': "text/css"})
    ## change reactions
    metadata.string = metadata.string.replace(".reaction-label{font-size:30px;fill:#202078;text-rendering:optimizelegibility}",
                                              ".reaction-label{"+f"font-size:{rxn_label_px}px;fill:#{rxn_label_hex};stroke:#000000;stroke-width:{rxn_label_px/20}px;"+"text-rendering:optimizelegibility}")
    if largeEdgeLabels is not None:
        fadedEdgesColor = tint_color(rxn_edge_hex, tint_factor)
        fadedEdgesLabelsColor = tint_color(rxn_label_hex, tint_factor)
        metadata.string = metadata.string + f" .m{fadedEdgesLabelsColor}-label"+"{"+f"font-size:{rxn_label_px/1.2}px;fill:#{fadedEdgesLabelsColor};stroke:#000000;stroke-width:{rxn_label_px/30}px;"+"text-rendering:optimizelegibility}"
        #f"stroke:#{fadedEdgesColor};"+f"stroke-width:{rxn_edge_px/1.2}px;"+"fill:none}"
    for reaction in soup.find_all(class_="reaction"):
        label = reaction.find("text")
        if largeEdgeLabels is not None and reaction["id"] not in largeEdgeLabels:
            label['class'] = f"m{fadedEdgesLabelsColor}-label"
        cleanedStr = label.string.split('_')[0].replace("-ABX", '').replace("RC", '').replace("WD", '')
        label.string = cleanedStr.split('.')[0][:6] + "." + cleanedStr.split('.')[1]
    ## change nodes
    metadata.string = metadata.string.replace(".node-label{font-size:20px}", 
                                              ".node-label{"+f"font-size:{node_label_px}px;fill:#{node_label_hex};stroke:#000000;stroke-width:{node_label_px/20}px;"+"}")
    metadata.string = metadata.string.replace(".metabolite-circle{stroke:#a24510;fill:#e0865b}",
                                              ".metabolite-circle{stroke:#"+f"{node_label_hex};fill:#{node_label_hex}"+"}")
    if largeNodeLabels is not None:
        fadedNodesColor = tint_color(node_label_hex, tint_factor)
        print(fadedNodesColor)
        for node in soup.find_all(class_="node"):
            # print(node)
            if node["id"] not in largeNodeLabels:
                # print(node["id"])
                circle = node.find("circle")
                label = node.find("text")
                if any([x is None for x in [circle, label]]):   continue
                circle["class"] = f"m{fadedNodesColor}"
                label['class'] = f"m{fadedNodesColor}-label"
                # print(node["id"])
        metadata.string = metadata.string + f" .m{fadedNodesColor}"+"{"+f"stroke:#{fadedNodesColor};fill:#{fadedNodesColor}"+"}"
        metadata.string = metadata.string + f" .m{fadedNodesColor}-label"+"{"+f"font-size:{node_label_px/1.2}px;fill:#{fadedNodesColor};stroke:#000000;stroke-width:{node_label_px/30}px;"+"}"
    ## change segments
    metadata.string = metadata.string.replace(".segment{"+"stroke:#334E75;stroke-width:10px;fill:none}", 
                                              ".segment{"+f"stroke:#{rxn_edge_hex};"+f"stroke-width:{rxn_edge_px}px;"+"fill:none}")
    # metadata.string = metadata.string.replace(".direction-arrow{stroke:#000;stroke-width:1px;fill:#fff;opacity:.3}",
    #                                           ".direction-arrow{stroke:#000;"+f"stroke-width:{rxn_edge_px*8}px;fill:#fff;opacity:{tint_factor}"+"}")
    # metadata.string = metadata.string.replace("arrowhead{fill:#334E75}",
    #                                           "arrowhead{fill:#000000"+f";stroke-width:{rxn_edge_px*8}px;"+"}")
    ## change miscellaneous text
    metadata.string = metadata.string.replace(".text-label-input{font-size:50px}", ".text-label-input{font-size:"+f"{rxn_label_px*1.8}"+"px}")
    # text['font-weight'] = 'bold'

    # optimize text locations
    # labelSizes = [node_label_px, rxn_edge_px]
    # adjust_labels(soup.find_all('text'), sum(labelSizes)/len(labelSizes))

    ## re-order the labels such that the text overlays all other elements
    # labels = soup.find_all('text')
    # edges = soup.find_all('reaction')  # Assuming edges are defined by <path> elements
    # nodes = soup.find_all('circle')  # Assuming nodes are defined by <circle> elements

    # # Create a new SVG content with reordered elements
    # new_svg = BeautifulSoup('<svg xmlns="http://www.w3.org/2000/svg"></svg>', 'lxml-xml')
    # new_svg.svg.attrs = soup.svg.attrs  # Preserve original SVG attributes

    # # Function to append elements to new SVG
    # def append_elements(elements):
    #     for elem in elements:
    #         # Find the parent <g> element if the element is nested
    #         parent = elem.find_parent('g')
    #         if parent and parent not in new_svg.svg:
    #             new_svg.svg.append(parent)
    #         elif elem not in new_svg.svg:
    #             new_svg.svg.append(elem)

    # # Append edges first
    # append_elements(edges)

    # # Append nodes second
    # append_elements(nodes)

    # # Append labels last
    # append_elements(labels)

    # # Write the modified SVG content to a new file with proper indentation
    # with open("output.svg", "w", encoding="utf-8") as file:
    #     file.write(new_svg.prettify())


    # add ASV nodes
    for edge in soup.find_all(class_="segment"):
        # stroke-dasharray="10,5"   to implement dash or solid edges conditionally, based on the directionality of the exchange
        coordinates = list(map(float, re.split(",| ", edge["d"].replace("M", "").replace("M", "").replace("C", "").replace("undefined", "100000000"))))
        if abs(coordinates[0]-coordinates[2]) < 30 and abs(coordinates[1]-coordinates[3]) < 3:
            # print(coordinates)
            rect = soup.new_tag("rect")
            rect['x'] = str(coordinates[0]+float("10734.555053710938")/2.03)
            rect['y'] = str(coordinates[1]+float("5315.0328369140625")/7.66)
            rect['width'] = "40"
            rect['height'] = "15"
            rect['fill'] = f"#{rxn_label_hex}"
            soup.svg.append(rect)
            # print(edge)
        # break
    # change the color of select edges
    if colorElements is not None:
        # baseline_metadata = metadata.string
        edgeColor = {}
        for color, lst in colorElements.items():
            edgeColor.update({x:color for x in lst})
        for edge in soup.find_all(class_="segment-group"):
            if edge["id"] in edgeColor:
                color = edgeColor[edge["id"]]
                for child in edge.find_all(class_="segment"):
                    child['class'] = f"m{color}"
                # for i, child in enumerate(edge.children):
                #     if "segment" in child:
                #         edge.children[i] = child.replace("segment", f"segment m{color}")

                # display([x for x in edge.children])
                # break
            elif largeEdgeLabels is not None:
                for child in edge.find_all(class_="segment"):
                    child['class'] = f"m{fadedEdgesColor}"
        for color, elements in colorElements.items():
            # ".segment{"+"stroke:#334E75;stroke-width:10px;fill:none}"
            metadata.string = metadata.string + f" .m{color}"+"{"+f"stroke:#{color};"+f"stroke-width:{rxn_edge_px*1.2}px;"+"fill:none}"
            if any(['r' in x for x in elements]):
                for ele in elements:
                    if 'r' in ele:
                        reaction = soup.find(attrs={"id": ele})
                        label = reaction.find("text")
                        label["class"] = f"m{color}-label"
                metadata.string = metadata.string + f" .m{color}-label"+"{"+f"font-size:{rxn_label_px}px;fill:#{color};stroke:#000000;stroke-width:{rxn_label_px/20}px;"+"text-rendering:optimizelegibility}"
            if any(['n' in x for x in elements]):
                for ele in elements:
                    if 'n' in ele:
                        node = soup.find(attrs={"id": ele})
                        circle = node.find("circle")
                        circle["class"] = f"m{color}-circle"
                        label = node.find("text")
                        label["class"] = f"m{color}-label"
                metadata.string = metadata.string + f" .m{color}-circle"+"{"+f"stroke:#{color};fill:#{color}"+"}"
                
        if largeEdgeLabels is not None:
            metadata.string = metadata.string + f" .m{fadedEdgesColor}"+"{"+f"stroke:#{fadedEdgesColor};"+f"stroke-width:{rxn_edge_px/1.2}px;"+"fill:none}"
    
    # change the color of select nodes


    with open(svg_path.replace(".svg", "_edited.svg"), "w", encoding="utf-8") as file:
        file.write(soup.prettify())

EscherSVG_processing("metabolite_focused_map_IDs.svg", node_label_px=90, rxn_label_px=90, rxn_edge_px=6,
                     node_label_hex="E36400", rxn_label_hex="cc0000", rxn_edge_hex="2C79BD",
                     largeNodeLabels=["n202", "n311", "n900", "n963", "n33", "n32", "n830", "n71", "n1322", "n1443", "n1672", "n1673",
                                      "n2066", "n2102", "n206", "n404", "n873", "n1090", "n1182", "n1746", "n2154", "n2154", "n1338", # RC
                                      "n539", "n1063", "n1282", "n1511", "n1576" # cellibiose
                                      "n1540", "n249",   # Melitose
                                      "n85", "n104"   # WD
                                      ],
                     largeEdgeLabels=["r10", "r23", "r48", "r50", "r49", "r41", "r3", "r60", "r64", "r65", "r73", "r68", "r72", "r88", "r86", "r83",
                                      "r95", "r108", "r106", "r107",
                                      "r31", "r53", "r63", "r75", "r76", "r99", "r77", "r79", "r121", "s1435", "s1009", "s1218",      # WD
                                      ],
                     abbrevIDs=cpdID_abbrv,
                     colorElements={"8E0FFF":["s903", "s295", "s444", "s906", "s928", "s900", "s913", "s67", "s1246", "s1150", "s1254", "s1238", "s1390",
                                              "s1370", "s1666", "s1637", "s1588", "s788", "s66", "s1589", "s1787", "s1777", "s387", "s915",
                                              "s1170", "s1186", "s1152", "s2024", "s2072", "s2042", "s2024", "s2085", "s1993"   # RC
                                              "s599", "s602", "s593", "s596", "s597", "s603", "s2195", "s1479", "s98"    # WD
                                           ],  # purple
                                 "11C911":["s191", "s195", "r10", "r3", "r68", "r20", "s383", "s823", "s829", "n873", "r80", "r122"],  # green
                                 "F15757":["s1304"]  # red
                                 },
                     msdb=msdb, tint_factor=0.6)

f3c199


### create a custom ID mapping

In [402]:
import xml.etree.ElementTree as et

svgContent = et.parse("metabolite_focused_map.svg")
display(svgContent)
root = svgContent.getroot()
# display(dir(root))
print(root.tag)
reactions = root.find("g", attrs={'id': 'reactions'})
# for child in root:
#     print(child.tag, child.attrib)
#     if child.tag == "{http://www.w3.org/2000/svg}g":
#         for child2 in child:
#             print(child2.tag, child2.attrib)
#             if child2.id == "reactions":
#                 for child3 in child2:
#                     print(child3.tag, child3.attrib)


content = root.get("escher-svg")
display(content)

{http://www.w3.org/2000/svg}svg


TypeError: 'attrs' is an invalid keyword argument for find()